In [ ]:
#librerías
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
sys.path.append(r"D:\Repositorios\geoloc2\preparacion_datos\src")
from preparacion_inter_puntos import prep

Lectura de la base de CURT correspondiente al municipio (Tipo shp)

In [ ]:
#base de curt y manzanas municipales
CURT_mun=gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\CURT\060 Ixtapan de la Sal\predios_060.shp")
manzanas_mun = gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\Full manzanas\Full manzanas\Manzana_Ixtapan_Sal.shp")
#reemplazar id cat por clave catastral
CURT_mun['CLAVECATASTRAL']= CURT_mun['id_cat']
manzanas_mun['CLAVECATASTRAL']= manzanas_mun['cve_cat']

In [ ]:
#revisar si hay vacíos en clave catastrar en CURT y Manzanas
manzanas_mun[manzanas_mun.CLAVECATASTRAL.isna()]
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

In [ ]:
#Revisar cuantas CC se duplican
CURT_mun.CLAVECATASTRAL.value_counts()

In [ ]:
#Disolución de las geometrías a partir de clavecatastral 
CURT_mun=CURT_mun.dissolve(by='CLAVECATASTRAL').reset_index()
CURT_mun

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs Manzanas: ",manzanas_mun.crs)

In [ ]:
#Se le asigna el crs de CURT a la base de manzanas
manzanas_mun=manzanas_mun.to_crs(4326)
# Se hace un overlay para la unión de las bases 
CURT_mun=CURT_mun.overlay(manzanas_mun, how='union',keep_geom_type=False)
CURT_mun

In [ ]:
CURT_mun['CLAVECATASTRAL_1']=CURT_mun.CLAVECATASTRAL_1.combine_first(CURT_mun.CLAVECATASTRAL_2)
CURT_mun

In [ ]:
def combine_first_drop(gdf):
    for i in (gdf.columns):
        if str('_1') in i :
            n = i
        if str('_2') in i :
            k = i
            z = i[:i.rfind('_1')-1]
            gdf[z] = gdf[n]
            gdf.drop(columns=[n,k],inplace=True)
        else:
            pass

In [ ]:
#Se utiliza esta función para renombrar columna con _1 y eliminar la otra
combine_first_drop(CURT_mun)
CURT_mun

In [ ]:
def separar_digitos(prueba):

    prueba.CLAVECATASTRAL = prueba.CLAVECATASTRAL.astype(str)
    for i in prueba.CLAVECATASTRAL:
        if str('000000') in i:
            cat = i
            cat=cat[:-6]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat
        elif str('00000000') in i :
            cat2=i
            cat2=cat2[:-8]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat2
        else:
            pass


In [ ]:
#Se utiliza esta función para separar CC en CC manzana y CC predio
separar_digitos(CURT_mun)
CURT_mun

In [ ]:
CURT_mun = CURT_mun.set_geometry('geometry')
CURT_mun['geometry']=CURT_mun.centroid

In [ ]:
CURT_mun['LATITUD']= CURT_mun['geometry'].y
CURT_mun['LONGITUD']= CURT_mun['geometry'].x
CURT_mun

In [ ]:
#Se vuelven a revisar
CURT_mun[CURT_mun.geometry.isna()]
# CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

In [ ]:
CURT_mun = CURT_mun.loc[~CURT_mun['manz'].astype(str).str.endswith('000')]
CURT_mun

In [ ]:
CURT_mun.to_file(r"D:\Secretaría\cruces_bases\Ixtapan/curt_Ixtapan.shp")

In [ ]:
base_predial_OD = pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan\Insumos\ANEXO 3 PRED DICIEMBRE 2022 - Arturo Baez.xlsx")
base_predial_OD

In [ ]:
#Se puede utilizar esta celda en caso de que la CC no este homologada a CLAVECATASTRAL
#base_predial.rename(columns={'nombre_CC': 'CLAVECATASTRAL'}, inplace=True)
#base_predial

In [ ]:
#Secrea una base con los registros que SI tienen CC en predial
base_predial=base_predial_OD[base_predial_OD.CLAVECATASTRAL.notna()]

In [ ]:
#Se crean la CC a nivel manzana y a nivel predio de la base predial 
base_predial['CLAVE_MANZ']=base_predial.CLAVECATASTRAL.str[0:8]+"00"
base_predial['CLAVE_PREDIO']=base_predial.CLAVECATASTRAL.str[0:10]
base_predial

In [ ]:
#quitar duplicados
base_predial.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
base_predial

In [ ]:
def delete_empty(df):

    for i in df.columns:
        df.loc[(df[i] =="") |(df[i] =="0") | (df[i] ==0) | (df[i].isna()) | (df[i] ==" " ), i]=float('Nan')
    df.dropna(axis=1, how='all', inplace=True)
    
    return df 

In [ ]:
delete_empty(base_predial)

In [ ]:
base_predial=base_predial[base_predial['EJERCICIO FISCAL'] == 2022]
base_predial

In [ ]:
base_predial.columns[base_predial.columns.isin(CURT_mun.columns)]
falta= base_predial.columns[~base_predial.columns.isin(CURT_mun.columns)]
falta

In [ ]:
cruce_cat=pd.merge(base_predial[['CLAVECATASTRAL', 'CLAVE_MANZ','CLAVE_PREDIO', 'No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL']],CURT_mun, left_on="CLAVE_PREDIO", right_on="CLAVE_CATASTRAL", how='inner')
cruce_cat

In [ ]:
base_predial_n = base_predial.loc[~base_predial['CLAVE_PREDIO'].isin(cruce_cat['CLAVE_PREDIO'])]
base_predial_n

In [ ]:
base_predial_n.columns[base_predial_n.columns.isin(CURT_mun.columns)]
falta= base_predial_n.columns[~base_predial_n.columns.isin(CURT_mun.columns)]
falta

In [ ]:
cruce_manzana = pd.merge(base_predial_n[['CLAVECATASTRAL', 'No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL', 'CLAVE_MANZ',
       'CLAVE_PREDIO']], CURT_mun, left_on='CLAVE_MANZ', right_on='CLAVE_CATASTRAL', how='inner')
cruce_manzana

In [ ]:
cruce_predial_manz = pd.concat([cruce_cat,cruce_manzana])
cruce_predial_manz

In [ ]:
no_cruzo_predialxmanz=base_predial.loc[~base_predial['CLAVECATASTRAL'].isin(cruce_predial_manz['CLAVECATASTRAL_x'])]
no_cruzo_predialxmanz.to_csv(r"D:\Secretaría\cruces_bases\Ixtapan/no_cruzo_predialxmanz.csv", encoding="utf-8-sig")

In [ ]:
cruce_predial_manz.columns

In [ ]:
cruce_predial_manz['CLAVECATASTRAL']=cruce_predial_manz.CLAVECATASTRAL_x.combine_first(cruce_predial_manz.CLAVECATASTRAL_y)
cruce_predial_manz[cruce_predial_manz.CLAVECATASTRAL.isna()]

In [ ]:
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_predial_manz.drop(['CLAVECATASTRAL_y','CLAVECATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_predial_manz.columns

In [ ]:
#Guardar base de predialvscurt
cruce_predial_manz.to_csv(r"D:\Secretaría\cruces_bases\Ixtapan/cruce_predial_curt.csv")

Lectura de las 3 bases de predial (Predial, propietario y deudores)

In [ ]:
pred_pred=pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PREDIAL.xlsx")
pred_pred

In [ ]:
pred_prop = pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PROPIETARIO.xlsx")
pred_prop

In [ ]:
pred_deu = pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan\Insumos\Ixtapan de La Sal_PREDIALADEUDO.xlsx")
pred_deu

In [ ]:
def delete_empty(df):

    for i in df.columns:
        df.loc[(df[i] =="") |(df[i] =="0") | (df[i] ==0) | (df[i].isna()) | (df[i] ==" " ), i]=float('Nan')
    df.dropna(axis=1, how='all', inplace=True)
    
    return df 

In [ ]:
#borrar columnas vacias
delete_empty(pred_prop)
delete_empty(pred_pred)
delete_empty(pred_deu)

In [ ]:
#borrar duplicados de las bases
pred_deu.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_pred.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_prop.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)

In [ ]:
pred_deu = pred_deu[pred_deu['EJERCICIO'] == 2021]
pred_deu

In [ ]:
pred_pred['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.str[0:10]
pred_prop['CLAVE_CATASTRAL']= pred_prop.CLAVECATASTRAL.str[0:10]
pred_deu['CLAVE_CATASTRAL']= pred_deu.CLAVECATASTRAL.str[0:10]

In [ ]:
#Asegurarse que se cortaron todas las claves y no hay nan
pred_pred[pred_pred.CLAVE_CATASTRAL.isna()]
pred_prop[pred_prop.CLAVE_CATASTRAL.isna()]
pred_deu[pred_deu.CLAVE_CATASTRAL.isna()]

Qué columnas están dentro de mi base x cruzar, que no están dentro de la 1ra base 

In [ ]:
pred_pred.columns[pred_pred.columns.isin(pred_prop.columns)]
falta= pred_pred.columns[~pred_pred.columns.isin(pred_prop.columns)]
falta

Se hace el cruce con las columnas que no están dentro de la 1er base + la llave

In [ ]:
#Cruce de predial completo con propietario (el resultado de este cruce es la primera parte de la base completa de predial, no debe tener duplicados por clave catastral unica a 16 digitos y esa es la key de cruce)
predial_general = pd.merge(pred_pred[['CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA']], pred_prop, on="CLAVECATASTRAL", how="inner")
predial_general

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
predial_general['CLAVE_CATASTRAL']=predial_general.CLAVE_CATASTRAL_x.combine_first(predial_general.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
predial_general = predial_general.drop(columns_to_drop, axis=1)
predial_general


In [ ]:
predial_general[predial_general.CLAVE_CATASTRAL.isna()]

In [ ]:
#revisión de columnas entre predial general (cruce 1) y predial adeudos
predial_general.columns[predial_general.columns.isin(pred_deu.columns)]
falta_deu= predial_general.columns[~predial_general.columns.isin(pred_deu.columns)]
falta_deu

In [ ]:
cruce_pred1=pd.merge(predial_general[['CLAVECATASTRAL','CLAVE_CATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR']],pred_deu, on="CLAVECATASTRAL",  how='outer', indicator=True)
cruce_pred1

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_pred1['CLAVE_CATASTRAL']=cruce_pred1.CLAVE_CATASTRAL_x.combine_first(cruce_pred1.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
cruce_pred1 = cruce_pred1.drop(columns_to_drop, axis=1)
cruce_pred1

In [ ]:
cruce_pred1[cruce_pred1.CLAVE_CATASTRAL.isna()]

In [ ]:
predial_completo = cruce_pred1[cruce_pred1['_merge'] =='both']
predial_sin_adeudos = cruce_pred1[cruce_pred1['_merge'] =='left_only']
predial_completo

In [ ]:
#guardar bases
predial_completo.to_csv(r'D:\Secretaría\cruces_bases\Ixtapan/predial_completo_Ixtapan.csv',encoding="utf-8-sig")
predial_sin_adeudos.to_csv(r'D:\Secretaría\cruces_bases\Ixtapan/predial_completo_Ixtapan_noadeudos.csv',encoding="utf-8-sig")

In [ ]:
#eliminar la columna de merge en la base a trabajar de predial completo
predial_completo=predial_completo.drop(['_merge'], axis=1)

In [ ]:
predial_completo.columns[predial_completo.columns.isin(cruce_predial_manz.columns)]
falta_pred_completo= predial_completo.columns[~predial_completo.columns.isin(cruce_predial_manz.columns)]
falta_pred_completo

In [ ]:
cruce_CURT_predial = pd.merge(predial_completo[['CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS']], cruce_predial_manz, on='CLAVE_CATASTRAL', how = 'outer', indicator=True)
cruce_CURT_predial

In [ ]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_CURT_predial['CLAVECATASTRAL']=cruce_CURT_predial.CLAVECATASTRAL_x.combine_first(cruce_CURT_predial.CLAVECATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVECATASTRAL_x', 'CLAVECATASTRAL_y']
cruce_CURT_predial = cruce_CURT_predial.drop(columns_to_drop, axis=1)
cruce_CURT_predial

In [ ]:
cruce_CURT_predial = cruce_CURT_predial.rename(columns = {'_merge' : 'cruce_predial_2_CURT'})

In [ ]:
#Revisar si la base tiene LAT y LON
cruce_CURT_predial[cruce_CURT_predial.LONGITUD.isna()]

Agregar LAT LON a los registros que no venían en la base de CURT

In [ ]:
cruce_CURT_predial.loc[cruce_CURT_predial['LATITUD'].isna(), 'LATITUD'] = 18
cruce_CURT_predial.loc[cruce_CURT_predial['LONGITUD'].isna(), 'LONGITUD'] = -99

In [ ]:
cruce_CURT_predial.drop("geometry",axis=1,inplace=True)

In [ ]:
cruce_CURT_predial = prep.transform_df_to_gpd(cruce_CURT_predial, 'LONGITUD','LATITUD', crs= 'EPSG:4326')
cruce_CURT_predial

In [ ]:
cruce_CURT_predial[cruce_CURT_predial.geometry.isna()]

In [ ]:
#exportar base 
# cruce_CURT_predial.to_excel(r"/Users/alfredoemiliomartinezcarbajal/Downloads/Atlacomulco_PREDIAL/cruce_curt_predial_Atlacomulco.xlsx")
cruce_CURT_predial.to_csv(r"D:\Secretaría\cruces_bases\Ixtapan/cruce_curt_predial_Ixtapan.csv",encoding='utf-8-sig',index=False)


In [ ]:
cruce_CURT_predial = pd.read_csv(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/cruce_predial_curt.csv",encoding='utf-8-sig')
cruce_CURT_predial

In [ ]:
# cruce_CURT_predial['ESTIMADO'] = cruce_CURT_predial['CLAVE_CATASTRAL'].value_counts().reset_index()['CLAVE_CATASTRAL']
base_temp=cruce_CURT_predial.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

In [ ]:
base_temp=base_temp.rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
base_temp

In [ ]:
crucexcurt=cruce_CURT_predial.merge(base_temp[['ESTIMADO','curt','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

In [ ]:
# crucexcurt.drop("curt_y",axis=1,inplace=True)
crucexcurt.rename(columns={'curt_y':'CURT', "curt_x":"curt"},inplace=True)
crucexcurt

In [ ]:
crucexcurt.to_csv(r'D:\Secretaría\cruces_bases\Ixtapan/Base_predial_PREGEO_Ixtapan.csv',encoding="utf-8-sig", index=False)

#Prep data

In [ ]:
geom_novacios=crucexcurt[crucexcurt.LATITUD != 18 ]

In [ ]:
test_igecem_final=CURT_mun[['geometry']].sjoin(geom_novacios)

In [ ]:
test_igecem_final.drop('cruce_predial_2_CURT',axis=1,inplace=True)

In [ ]:
#En el caso de que la base tenga alguna columna con acentos, se tiene que cambiar o no se podrá guardar como shp
test_igecem_final.rename(columns={'USO DE ÁREA HOMOGÉNEA':'USO DE AREA HOMOGÉNEA'},inplace=True)

In [ ]:
test_igecem_final.to_file(r"D:\Secretaría\cruces_bases\Ixtapan/test_igecem_final_Ixtapan.shp").drop(columns=['FECHAALTA','EJERCICIO FISCAL'])

In [ ]:
test_igecem_final.columns

In [ ]:
cols_malas = gpd.read_file(r"D:\Secretaría\cruces_bases\Ixtapan/test_igecem_final_Ixtapan.shp")


In [ ]:
cols_malas.columns